In [16]:
%pip install crewai 'crewai[tools]' groq tavily-python --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
import warnings
warnings.filterwarnings('ignore')

from crewai import Agent, Task, Crew, LLM
import os
from dotenv import load_dotenv

load_dotenv()

llm = LLM("groq/llama-3.1-8b-instant", temperature=0)

In [7]:
from crewai_tools import TXTSearchTool

with open("../health_data.txt", "r") as f:
    content = f.read()
    
txt_tool = TXTSearchTool(
    txt= content,
)

In [8]:
#https://github.com/zinyando/crewai_tavily_tool_demo/blob/main/src/crewai_tavily_tool_demo/crew.py code for this custom tool
from crewai_tools import BaseTool
from pydantic import BaseModel, Field
from pydantic import Field
from tavily import TavilyClient
from typing import Annotated, Optional, Any, Type


class TavilySearchInput(BaseModel):
    query: Annotated[str, Field(description="The search query string")]
    max_results: Annotated[
        int, Field(description="Maximum number of results to return", ge=1, le=10)
    ] = 5
    search_depth: Annotated[
        str,
        Field(
            description="Search depth: 'basic' or 'advanced'",
            choices=["basic", "advanced"],
        ),
    ] = "basic"


class TavilySearchTool(BaseTool):
    name: str = "Tavily Search"
    description: str = (
        "Use the Tavily API to perform a web search and get AI-curated results."
    )
    args_schema: Type[BaseModel] = TavilySearchInput
    client: Optional[Any] = None

    def __init__(self, api_key: Optional[str] = None, **kwargs):
        super().__init__(**kwargs)
        self.client = TavilyClient(api_key=api_key)

    def _run(self, query: str, max_results=5, search_depth="basic") -> str:
        if not self.client.api_key:
            raise ValueError("TAVILY_API_KEY environment variable not set")

        try:
            response = self.client.search(
                query=query, max_results=max_results, search_depth=search_depth
            )
            return self._process_response(response)
        except Exception as e:
            return f"An error occurred while performing the search: {str(e)}"

    def _process_response(self, response: dict) -> str:
        if not response.get("results"):
            return "No results found."

        results = []
        for item in response["results"][:5]:
            title = item.get("title", "No title")
            content = item.get("content", "No content available")
            url = item.get("url", "No URL available")
            results.append(f"Title: {title}\nContent: {content}\nURL: {url}\n")

        return "\n".join(results)

tavily_search = TavilySearchTool()

In [9]:
health_agent = Agent(
    role="Smart Health Context Analyzer",
    goal="Determine if query is user-specific  (e.g., contains 'my condition,' 'for me') and if necessary provide user medical context",
    tools = [txt_tool],
    verbose=True,
    backstory=(
        "You analyze patient data"
        "to determine if any relevant information exists for the query."
        "However, only do so if the query is user-specific  (e.g., contains 'my condition,' 'for me')"
    ),
    llm=LLM(model="gpt-4o", temperature=0),
)

In [10]:
search_agent = Agent(
    role="Senior Web Search Researcher",
    goal="Search trusted medical websites for relevant information for the query.",
    tools = [tavily_search],
    verbose=True,
    backstory=(
        "You are skilled at finding reliable medical information from trusted websites that end with .gov, .edu, and .org."
        "An expert web researcher that uses the web extremely well"
    ),
    llm=llm
)

In [11]:
synthesis_agent = Agent(
    role="Professional Medical Information Synthesizer",
    goal="Combine results from web searches, health data, and other sources into a coherent response to the query",
    tools = [],
    verbose=True,
    backstory=(
        "You synthesize data from multiple sources to create clear and concise answers for the user."
    ),
    llm=LLM(model="gpt-4o", temperature=0)
)

In [12]:
health_context_task = Task(
    description=(
        "Check if the query {query} EXPLICITLY references the user's health (e.g., contains 'my condition,' 'for me')"
        "If NO, then return '' and ignore the rest of task. If YES, proceed to the following steps: "
        "Using the tool, Provide relevant medical context to help answer the query"
        "Ensure there's no hallucination and include citations for each piece of information"
    ),
    expected_output=(
        "A structured health context report."
    ),
    agent=health_agent,
    async_execution=True
)

In [13]:
search_task = Task(
    description=(
        "Search trusted medical websites for information about: {query}\n"
        "Use the search_tool to retrieve information from the web"
        "Your task:"
        "1. Search trusted medical websites using: (site:.gov OR site:.edu OR site:.org)\n"
        "3. Return your results"
    ),
    expected_output=(
        "A structured report from medical sources with clear citations."
    ),
    agent=search_agent,
    async_execution=True
)

In [14]:
synthesis_task = Task(
    description=(
        "Create a comprehensive answer for: {query}"
        "Using:"
        "Health Context (if available)"
        "Web Search Results"
        "List all sources at the end, no need to include references"
        "Include a brief medical disclaimer"
        "Keep the response clear, and well-structured."
    ),
    expected_output=(
        "A clear medical response with proper organization and citations with links."
    ),
    context=[health_context_task, search_task],
    agent=synthesis_agent
)

In [15]:
health_crew = Crew(
    agents=[health_agent,
            search_agent,
            synthesis_agent,
    ],
    tasks=[health_context_task,
           search_task,
           synthesis_task,
    ],
    verbose=True
)

In [48]:
input = {'query':'What is an astrocytoma?'}
result = health_crew.kickoff(inputs=input)
from IPython.display import Markdown
Markdown(str(result))

# Agent: Smart Health Context Analyzer
## Task: Check if the query What is an astrocytoma? EXPLICITLY references the user's health (e.g., contains 'my condition,' 'for me')If NO, then return '' and ignore the rest of task. If YES, proceed to the following steps: Using the tool, Provide relevant medical context to help answer the queryEnsure there's no hallucination and include citations for each piece of information
# Agent: Senior Web Search Researcher
## Task: Search trusted medical websites for information about: What is an astrocytoma?
Use the search_tool to retrieve information from the webYour task:1. Search ONLY trusted medical websites using: (site:.gov OR site:.edu OR site:.org)
3. Return your results


# Agent: Smart Health Context Analyzer
## Final Answer: 





# Agent: Senior Web Search Researcher
## Using tool: Tavily Search
## Tool Input: 
"{\"query\": \"astrocytoma site:.gov OR site:.edu OR site:.org\", \"max_results\": 10, \"search_depth\": \"advanced\"}"
## Tool Outpu

Astrocytoma is a type of brain tumor that originates from astrocytes, which are star-shaped glial cells in the brain and spinal cord. It is a subtype of glioma, a category of tumors that arise from glial cells. Astrocytomas are classified based on their grade, which reflects how abnormal the tumor cells appear under a microscope and how quickly they are likely to grow.

**Classification and Grades:**
Astrocytomas are categorized into different grades, which help determine the treatment approach and prognosis:
- **Grade 1 (Pilocytic Astrocytoma):** These are often considered benign and are slow-growing. Surgery can often cure these tumors if they can be completely removed.
- **Grade 2 (Diffuse Astrocytoma):** These are low-grade tumors that grow slowly but can become more aggressive over time.
- **Grade 3 (Anaplastic Astrocytoma):** These tumors are malignant and grow more rapidly, often requiring additional treatments beyond surgery.
- **Grade 4 (Glioblastoma):** This is the most aggressive form of astrocytoma, known for rapid growth and a poor prognosis.

**Symptoms:**
The symptoms of astrocytoma can vary depending on the tumor's location and size. Common symptoms include:
- Headaches
- Seizures
- Nausea and vomiting
- Personality changes
- Neurological deficits such as weakness or sensory changes

**Causes and Risk Factors:**
While the exact cause of astrocytomas is not fully understood, certain genetic conditions, such as tuberous sclerosis complex (TSC), can increase the risk of developing these tumors. Environmental factors and genetic mutations may also play a role.

**Diagnosis and Treatment:**
Diagnosis typically involves imaging studies such as MRI or CT scans, followed by a biopsy to determine the tumor's grade. Treatment options depend on the tumor's grade and location and may include:
- Surgery: Often the first step to remove as much of the tumor as possible.
- Radiation therapy: Used to target and kill remaining cancer cells.
- Chemotherapy: May be used in conjunction with radiation, especially for higher-grade tumors.

**Prognosis:**
The prognosis for astrocytoma varies widely based on the tumor's grade and the patient's overall health. Lower-grade tumors generally have a better prognosis, while higher-grade tumors like glioblastomas are more challenging to treat effectively.

**Medical Disclaimer:**
This information is provided for educational purposes only and is not intended as medical advice. For specific medical concerns, please consult a healthcare professional.

Sources:
- UCSF Brain Tumor Center
- Mayo Clinic
- Stanford Medicine Brain Tumor Center
- National Center for Biotechnology Information (NCBI) Bookshelf
- Cleveland Clinic